In [1]:
import numpy as np
import os
import torch

import torchvision.models as models

import copy

random_seed = 4545435
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
device

device(type='cpu')

In [5]:
#  Создает в директории csv файл, размечающий картинки
def generate_csv_classes(file_name):
    file_csv = open(file_name, "w+")
    count_class = 0
    res = "filename, full_filename, num_class\n"
    for i in os.listdir("../resized_images"):
        for j in os.listdir("../resized_images"+os.sep+i):

            res += "{},{},{}\n".format(j.split(".")[0], "../resized_images"+os.sep+i+os.sep+j, count_class)
        count_class+=1
           
    file_csv.write(res)
    file_csv.close()
generate_csv_classes("sample_full.csv")

In [2]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd

class MyCustomDataset(Dataset):
    def __init__(self, dict_name):
        self.transformations = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        self.class_dict = pd.read_csv(dict_name)
        self.data_len = len(self.class_dict.index)
        
    def __getitem__(self, index):
        res_row = self.class_dict.iloc[index]
        full_filename = res_row[1]
        data = Image.open(full_filename).convert('RGB').resize((224, 224))
        data = self.transformations(data)  # (3)
        label = res_row[2]
        return (data, label)

    def __len__(self):
        
        return self.data_len

In [3]:
data = MyCustomDataset("sample_full.csv")

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler 
dataset_size = len(data) 
indices = list(range(dataset_size)) 
validation_split = 0.4
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed) 
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices) 
valid_sampler = SubsetRandomSampler(val_indices)
BATCH_SIZE = 64
train_loader = DataLoader(data, batch_size = BATCH_SIZE,  num_workers=0, sampler=train_sampler)

val_loader = DataLoader(data, batch_size = BATCH_SIZE, num_workers=0, sampler=valid_sampler)
print(len(train_loader))

293


In [5]:
device

device(type='cpu')

In [6]:
cnn = models.vgg19(pretrained=True).features.to(device).eval() #

In [7]:
def gram_matrix(inp):
    a, b, c, d = inp.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = inp.view(a * b, c * d).to(device)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [8]:
import torch.nn as nn
import torch.nn.functional as F


cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std
    

In [9]:
class StyleMatrix(nn.Module):

    def __init__(self):
        super(StyleMatrix, self).__init__()

    def forward(self, inp):
        G = gram_matrix(inp)
        return G

In [12]:
model = copy.deepcopy(cnn)
model = torch.nn.Sequential(*(list(model.children())[:26])+[StyleMatrix()])
print(model)  # Сама сетка (можно посчитать, что последний conv - тот, который нужен)
torch.save(model.state_dict(), "cutted_vgg19.pth")

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

### Отбор признаков (можно пропустить)
Каждую итерацию исключается ровно половина признаков, выявление наиболее важных происходит на 500, 1000, ... объектах в зависимости от текущего количества неисключенных признаков. В конце остаётся ~4000, которые можно сохранить в csv или продолжить работу.

In [169]:
from sklearn.feature_selection import SelectKBest, chi2

def select_features(mask, number, k):
#     inds = np.random.choice(len(data), size=min(len(data), number), replace=False)
    X = []
    y = []
    for i, (images, labels) in enumerate(train_loader):
        G_matrix = model(torch.reshape(images, (1, 3, 224, 224)).to(device))
        X.append((G_matrix.to(cpu).detach().numpy().reshape((1,512*512))[0][mask] + 1) / 2)
        y.append(labels)
        if i > min(len(data), number):
            break
    selector = SelectKBest(chi2, k=k)
    selector.fit(X, y)
    return selector.get_support(indices=True)

In [161]:
import time
start_time = time.time()
mask_history = []

mask = np.array([i for i in range(262144)])
while len(mask) > 10000:
    print(500 * 262144 // len(mask), 'objects')
    mask = mask[[list(select_features(mask, 500 * 262144 // len(mask), len(mask) // 2))]]
    mask_history.append(mask)
    print('n_features =', len(mask))
    print((time.time() - start_time), 'seconds spent')

500 objects


<ipython-input-161-49a478c31b54>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask = mask[[list(select_features(data, mask, 500 * 262144 // len(mask), len(mask) // 2))]]


n_features = 131072
36.633594036102295 seconds spent
1000 objects


<ipython-input-161-49a478c31b54>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask = mask[[list(select_features(data, mask, 500 * 262144 // len(mask), len(mask) // 2))]]


n_features = 65536
135.84219074249268 seconds spent
2000 objects


...

<ipython-input-161-49a478c31b54>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask = mask[[list(select_features(data, mask, 500 * 262144 // len(mask), len(mask) // 2))]]


n_features = 16384
465.3714876174927 seconds spent
8000 objects


<ipython-input-161-49a478c31b54>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask = mask[[list(select_features(data, mask, 500 * 262144 // len(mask), len(mask) // 2))]]


n_features = 8192
905.3595354557037 seconds spent
16000 objects


KeyboardInterrupt: 

Сохраним номера признаков на разных шагах в файл на случай их утери.

In [212]:
len(mask_history[-1])

8192

In [233]:
from numpy import asarray, save

save('features65000.npy', asarray(mask_history[-4]))
save('features32000.npy', asarray(mask_history[-3]))
save('features16000.npy', asarray(mask_history[-2]))
save('features8000.npy', asarray(mask_history[-1]))

### Отсюда начинать при отсутствии CSV файлов

Загрузить сохранённые данные:

In [18]:
from numpy import load

mask = load('features8000.npy')
len(mask)

8192

Получим данные и сохраним их в CSV.

In [19]:
def get_dataset(loader, features):
    X = []
    y = []
    for i, (images, labels) in enumerate(loader):
        G_matrix = model(torch.reshape(images, (1, 3, 224, 224)).to(device))
        X.append(G_matrix.to(cpu).detach().numpy().reshape((1,512*512))[0][features])
        y.append(labels)
        if (i % 1000) == 0:
            print(i, 'out of', len(loader), 'completed')
    df = pd.DataFrame(data=X)
    df['label'] = int(y)
    return df

In [ ]:
df_train = get_dataset(train_loader, mask)

0 out of 18735 completed
1000 out of 18735 completed
2000 out of 18735 completed
3000 out of 18735 completed
4000 out of 18735 completed
5000 out of 18735 completed
6000 out of 18735 completed
7000 out of 18735 completed
8000 out of 18735 completed
9000 out of 18735 completed
10000 out of 18735 completed


In [199]:
df_train.to_csv('train.csv')

In [196]:
df_val = get_dataset(val_loader, mask)

0 out of 12490 completed
1000 out of 12490 completed
2000 out of 12490 completed
3000 out of 12490 completed
4000 out of 12490 completed
5000 out of 12490 completed
6000 out of 12490 completed
7000 out of 12490 completed
8000 out of 12490 completed
9000 out of 12490 completed
10000 out of 12490 completed
11000 out of 12490 completed
12000 out of 12490 completed


In [197]:
df_val.to_csv('val.csv')

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.feature_selection import SelectKBest, chi2

# for (images, labels) in data:
#     G_matrix = model(torch.reshape(images, (1, 3, 224, 224)).to(device))
#     trans = transforms.ToPILImaMge()
#     plt.imshow(trans(G_matrix))
#     plt.show()
#     print(G_matrix.to(cpu).detach().numpy().reshape((1,512*512))[0].shape)
#     print(np.array(G_matrix))
#     break
# #     trans = transforms.ToPILImage()
# #     plt.imshow(trans(G_matrix))
# #     plt.show()